## Importing all the library here.

In [1]:
import pandas as pd
import numpy as np
import warnings as w
import requests 
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
import random
from sentence_transformers import SentenceTransformer

In [2]:
data =pd.read_csv('data.csv')

In [3]:
pd.set_option('display.max_columns', None)    
pd.set_option('display.max_colwidth', None)

In [4]:
data.head()

,Sentence,Sentiment
0,"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .",positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real possibility",negative
2,"For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .",positive
3,"According to the Finnish-Russian Chamber of Commerce , all the major construction companies of Finland are operating in Russia .",neutral
4,"The Swedish buyout firm has sold its remaining 22.4 percent stake , almost eighteen months after taking the company public in Finland .",neutral


In [5]:
data['Sentiment'].value_counts()

Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64

In [6]:
data.shape

(5842, 2)

## Removing the duplicates

In [7]:
data.duplicated().sum()

6

In [8]:
data.drop_duplicates(inplace = True)

In [9]:
data.shape,data.size

((5836, 2), 11672)

## Encode Sentiments


In [10]:
label_encoder = LabelEncoder()
data['Sentiment_Label'] = label_encoder.fit_transform(data['Sentiment'])

In [11]:
data.head(2)

,Sentence,Sentiment,Sentiment_Label
0,"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .",positive,2
1,"$ESI on lows, down $1.50 to $2.50 BK a real possibility",negative,0


## Vectorize text using TF-IDF


In [12]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['Sentence']).toarray()
y = data['Sentiment_Label'].values

In [13]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
y

array([2, 0, 2, ..., 1, 1, 2])

## train test split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4668, 5000)
(1168, 5000)
(4668,)
(1168,)


## RL Environment

In [17]:
class SentimentEnv:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.index = 0

    def reset(self):
        self.index = 0
        return self.X[self.index]

    def step(self, action):
        reward = 1 if action == self.y[self.index] else -1
        self.index += 1
        done = self.index >= len(self.X)
        state = None if done else self.X[self.index]
        return state, reward, done

## Policy Network


In [18]:
class PolicyNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNet, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return torch.softmax(self.fc(x), dim=-1)

## Initialize environment and model


In [19]:
env = SentimentEnv(X_train, y_train)
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
policy_net = PolicyNet(input_dim, output_dim)
optimizer = optim.Adam(policy_net.parameters(), lr=0.01)

## Training with REINFORCE



In [20]:
epochs = 5
for epoch in range(epochs):
    state = env.reset()
    log_probs = []
    rewards = []
    while True:
        state_tensor = torch.FloatTensor(state)
        probs = policy_net(state_tensor)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        log_probs.append(log_prob)

        next_state, reward, done = env.step(action.item())
        rewards.append(reward)

        if done:
            break
        state = next_state

    # Compute loss
    loss = -torch.stack(log_probs) * torch.FloatTensor(rewards)
    loss = loss.sum()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Total reward: {sum(rewards)}")

Epoch 1, Total reward: -1438
Epoch 2, Total reward: -1500
Epoch 3, Total reward: -1534
Epoch 4, Total reward: -1188
Epoch 5, Total reward: -1326


## Evaluate


In [21]:
policy_net.eval()
preds = []
for x in X_test:
    with torch.no_grad():
        probs = policy_net(torch.FloatTensor(x))
        pred = torch.argmax(probs).item()
        preds.append(pred)

In [22]:
print("Accuracy:", accuracy_score(y_test, preds))
print("F1 Score:", f1_score(y_test, preds, average='weighted'))

Accuracy: 0.6087328767123288
F1 Score: 0.4975541549983422
